In [1]:
import duckdb
db = duckdb.connect()
db.install_extension("h3", repository = "community")
db.close()

In [2]:
import ibis
from ibis import _
con = ibis.duckdb.connect(extensions=['httpfs', 'spatial', 'h3'])


In [3]:
# enable ibis to use built-in function from the h3 extension
@ibis.udf.scalar.builtin
def h3_cell_to_boundary_wkt	(array) -> str:
    ...

In [4]:
# Configure write-access to source.coop
import streamlit as st
source_key = st.secrets["SOURCE_KEY"]
source_secret = st.secrets["SOURCE_SECRET"]

query=   f'''
CREATE OR REPLACE SECRET secret1 (
    TYPE S3,
    KEY_ID '{source_key}',
    SECRET '{source_secret}',
    ENDPOINT 'data.source.coop',
    URL_STYLE 'path'

);
'''

minio_key = st.secrets["MINIO_KEY"]
minio_secret = st.secrets["MINIO_SECRET"]
query=   f'''
CREATE OR REPLACE SECRET secret2 (
    TYPE S3,
    KEY_ID '{minio_key}',
    SECRET '{minio_secret}',
    ENDPOINT 'minio.carlboettiger.info',
    URL_STYLE 'path'

);
'''

con.raw_sql(query)

In [37]:
gbif = con.read_parquet("s3://cboettig/gbif/2024-10-01/**")

In [18]:
%%time
(gbif
  .filter(_["class"].isin(["Insecta"]))
#  .filter(_["class"].isin(["Mammalia", "Insecta", "Aves", "Reptilia", "Amphibia"])
 # .filter(_.species == "Canis latrans")
  .rename(hex = "h5")
  .select(_.hex, _.species)
  .distinct()
  .group_by(_.hex)
  .agg(n = _.count())
#  .mutate(wkt =  h3_cell_to_boundary_wkt(_.hex))
  .mutate(logn = _.n.log())
  .mutate(value = (255 * _.logn / _.logn.max()).cast("int"))
  .to_csv("s3://public-gbif/csv/gbif_hex_insect_richness.csv")
)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 4min 27s, sys: 1min 4s, total: 5min 31s
Wall time: 16 s


In [6]:
import pydeck as pdk
import pandas as pd

COUNTRIES = "https://d2ad6b4ur7yvpq.cloudfront.net/naturalearth-3.3.0/ne_50m_admin_0_scale_rank.geojson"
GBIF = "https://data.source.coop/cboettig/gbif/csv/gbif_hex_mammal_richness_z3.csv"

GBIF = "https://minio.carlboettiger.info/public-gbif/csv/gbif_hex_insect_richness.csv"

view_state = pdk.ViewState(latitude=51.47, longitude=0.45, zoom=0)
view = pdk.View(type="_GlobeView", controller=True, width=1000, height=600)


layers = [
    pdk.Layer(
        "GeoJsonLayer",
        id="base-map",
        data=COUNTRIES,
        stroked=False,
        filled=True,
        get_fill_color=[200, 200, 200],
    ),
    pdk.Layer(
        "H3HexagonLayer",
        id="gbif",
        data=GBIF,
        extruded=True,
        get_elevation="value",
        get_hexagon="hex",
        elevation_scale=50,
        elevation_range = [0,1],
        pickable=True,
        auto_highlight=True,
        get_fill_color="[255 - value, 255, value]",
    ),
]

deck = pdk.Deck(
    views=[view],
    initial_view_state=view_state,
    layers=layers,
    map_provider=None,
    # Note that this must be set for the globe to be opaque
    parameters={"cull": True},
)

deck.to_html("gbif.html")

# MapLibre

In [38]:

## Filter to specific polygon (CA)
import geopandas as gpd
states = gpd.read_file("https://github.com/nvkelso/natural-earth-vector/raw/master/geojson/ne_110m_admin_1_states_provinces.geojson")
ca_polygon = states[states.name == 'California']
ca_poly_expr = ibis.literal(ca_polygon.geometry.iloc[0])


name = "ca_bird_richness"
s3_csv = "s3://public-gbif/csv/" + name + ".csv"

subset = (gbif
  .filter( _.geom.within(ca_poly_expr))
  .filter(_["class"] == "Aves")
  .rename(hex = "h10")
  .select(_.hex, _.species)
  .distinct()
  .group_by(_.hex)
  .agg(n = _.count())
#  .mutate(wkt =  h3_cell_to_boundary_wkt(_.hex))
  .mutate(logn = _.n.log())
  .mutate(value = (255 * _.logn / _.logn.max()).cast("int"))
  .to_csv(s3_csv)
)





FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [29]:
# maplibre wants json, not csv, shuffle shuffle
import minio
client = minio.Minio("minio.carlboettiger.info", minio_key, minio_secret)
con.read_csv(s3_csv).execute().to_json(name + ".json", orient='records', indent=2)
client.fput_object("public-gbif", "csv/" + name + ".json", name + ".json")

In [35]:
import leafmap.maplibregl as leafmap
import pydeck as pdk
m = leafmap.Map(center=[-121.9, 39.3], zoom=6, style="positron")

url = "https://minio.carlboettiger.info/public-gbif/csv/ca_bird_richness.json"


source = {
    "type": "geojson", 
    'data': 'https://data.source.coop/cboettig/us-boundaries/mappinginequality.json',
  #  'filter': ['==', ['get', 'city'], 'New Haven']
}
redlining = {
    "id": "redlining",
    "type": "fill",
    "source": "geojson",
     'paint': {
        "fill-color": ["get", "fill"], # color by the column called "fill"
        "fill-opacity": 0.8,
    },
}

text_layer = {
    "id": "labels",
    "type": "symbol",
    "source": "geojson",
    "layout": {
        "text-field": ["get", "label"],
        "text_size": 14,
        'text-font': ['Open Sans Bold'],
        "text-anchor": "center",
    },
     'paint': { 'text-color': '#ffffff'}
}


gbif =  pdk.Layer(
        "H3HexagonLayer",
        id="gbif",
        data=url,
        extruded=True,
        get_elevation="value",
        get_hexagon="hex",
        elevation_scale=20,
        elevation_range = [0,1],
        pickable=True,
        auto_highlight=True,
        get_fill_color="[255, 255 - value, value]",
    )

#m.add_basemap("Esri.NatGeoWorldMap")
m.add_source("geojson", source)
m.add_layer(redlining)
m.add_layer(text_layer)
m.add_deck_layers([gbif])
m.add_layer_control()

m.to_html("ca_bird_richness.html", overwrite=True)

m

Map(height='600px', map_options={'bearing': 0, 'center': (-121.9, 39.3), 'pitch': 0, 'style': 'https://basemap…

In [26]:
import pydeck as pdk
csv_url = "https://minio.carlboettiger.info/public-biodiversity/test.csv"
layer = pdk.Layer(
        "H3HexagonLayer",
        id="gbif",
        data=csv_url,
        extruded=True,
        get_elevation="count",
        get_hexagon="hex",
        elevation_scale=50,
        elevation_range = [0,1],
        pickable=True,
        auto_highlight=True,
        get_fill_color="[255 - count, 255, count]",
    ),

view_state = pdk.ViewState(latitude=37.74, longitude=-122.4, zoom=10)
pdk.Deck(initial_view_state = view_state, layers = [layer])



{
  "initialViewState": {
    "latitude": 37.74,
    "longitude": -122.4,
    "zoom": 10
  },
  "layers": [
    [
      {
        "@@type": "H3HexagonLayer",
        "autoHighlight": true,
        "data": "https://minio.carlboettiger.info/public-biodiversity/test.csv",
        "elevationRange": [
          0,
          1
        ],
        "elevationScale": 50,
        "extruded": true,
        "getElevation": "@@=count",
        "getFillColor": "@@=[255 - count, 255, count]",
        "getHexagon": "@@=hex",
        "id": "gbif",
        "pickable": true
      }
    ]
  ],
  "mapProvider": "carto",
  "mapStyle": "https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json",
  "views": [
    {
      "@@type": "MapView",
      "controller": true
    }
  ]
}

In [27]:
from maplibre import MapOptions
from maplibre.basemaps import Carto
from maplibre.controls import NavigationControl
from maplibre.ipywidget import MapWidget as Map
m = Map(MapOptions(center=(-122.4, 37.74), zoom=10), controls=[NavigationControl()])
m.add_deck_layers([layer])
m

MapWidget(height='400px', map_options={'center': (-122.4, 37.74), 'style': 'https://basemaps.cartocdn.com/gl/d…